# Functional Programming

![Alt](evolution.png "Evolution")

## Benefits

### Simplicity and Legibility

#### With Boilerplate

```javascript
var things = [1, 2, 3, 4];
for (var i = 0; i < things.length; ++i) {
    things[i] = things[i] * 10;
}
console.log(things);
```

#### No Boilerplate

In [88]:
val things = List(1, 2, 3, 4)

val newThings = things.map(v => v * 10)

things: List[Int] = List(1, 2, 3, 4)
newThings: List[Int] = List(10, 20, 30, 40)

#### With Boilerplate

```java
interface Calc {
  int operation(n, m):Integer;
}

class CalcAdds implements Calc {
  int operation(n, m) { return n + m }
}

class CalcMult implements Calc {
  int operation(n, m:Integer) { return n * m }
}

Calc sum = new CalcSum()
int result = sum.operation(1,2)

Calc mult = new CalcMult()
int result = mult.operation(1,2)
```

#### No Boilerplate

In [49]:
object Calc {
	def calc(a:Int,b:Int,f:(Int,Int)=>Int):Int = {
   	 f(a,b)
	}
}

val resultSum = Calc.calc(1,3,(_+_))

val resultMul = Calc.calc(1,3,(_*_))

defined object Calc
resultSum: Int = 4
resultMul: Int = 3

### Composibility

In [50]:
def twofold(n:Int) = {n * 2}

def threefold(n:Int) = {n * 3}

val sixfold = twofold _ compose threefold _

sixfold(2)

defined function twofold
defined function threefold
sixfold: Int => Int = <function1>
res49_3: Int = 12

### Debugability

```java
int rt(int x)
{
  return x + 1;
}
```

This is easy to reason about: 

```java
rt(6) = 6 + 1 = 7
```

```java
int globalValue = 0;

int rq(int x)
{
  globalValue++;
  return x + globalValue;
}
```


This is hard to reason about: 

```java
rq(6) = ???
```

What about the following?

```java
rq(x) + rq(y) * (rq(x) - rq(x))
```

### Quality and Robustness

Immutability, legibility, simplicity and debugability makes the code less errorprone and easier to test.

### Concurrency and Parallelism

Immutability along with Futures and Promises help avoiding common concurrency issues like race conditions and deadlocks.

### Higher Level Abstractions

Functional Programming pushes us towards thinking in terms of higher level abstractions, i.e., extract common behaviour duplicated throughout the code into higher level functions.

The most common functional abstractions are:

**map:**

```scala
(1 to 5).map(_*2)
//Vector(2, 4, 6, 8, 10)
```

**flatMap:**

```scala
(1 to 5).flatMap(1 to _)
//Vector(1, 1, 2, 1, 2, 3, 1, 2, 3, 4, 1, 2, 3, 4, 5)
```

**reduce:**

```scala
(1 to 5).reduce(_+_)
//15
```

## Basic Concepts

### REPL

REPL stands for read, eval, print, loop. 

Most functional programming languages provide a REPL which can be used for rapid experimentation and prototyping.

### Anonymous Functions

Anonymous functions help increasing legibility by reducing the need for boilerplate code.

In [82]:
val f = (x:Int, y:Int)=> x + y

(1 to 5).map((x) => x + 1)

f: (Int, Int) => Int = <function2>
res81_1: collection.immutable.IndexedSeq[Int] = Vector(2, 3, 4, 5, 6)

### Currying and Partial Function Application

Currying is a method of producing a chain of functions from a function with multiple arguments. 

The arguments of the original function will be split into the new generated functions.

Given the mod function below:

In [52]:
def mod(n:Int, x:Int) = ((x % n) == 0)

defined function mod

We can define a curried version of mod like in the code snippet below:

In [83]:
def modCurried(n: Int): (Int) => Boolean = {
  (x:Int) => mod(n, x)
}

defined function modCurried

In [85]:
def filter(xs: List[Int], p: Int => Boolean): List[Int] = {
  if (xs.isEmpty) xs
  else if (p(xs.head)) xs.head :: filter(xs.tail, p)
  else filter(xs.tail, p)
}

val nums = List(1, 2, 3, 4, 5, 6, 7, 8)

filter(nums, modCurried(2))
filter(nums, modCurried(3))

defined function filter
nums: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8)
res84_2: List[Int] = List(2, 4, 6, 8)
res84_3: List[Int] = List(3, 6)

Uncurrying is the inverse of currying, i.e, we generate a function with more arguments than the originals functions.

In [54]:
def modUncurried(n: Int, x:Int) = {
	modCurried(n)(x)
}

defined function modUncurried

Currying enables Partial function application, which allows a function to have some of its parameters fixed, generating a new function with lower arity. 

For example:

In [55]:
def add(x: Int, y: Int) = {x + y} 

val inc = add(1, _: Int)

val res = inc(0)

defined function add
inc: Int => Int = <function1>
res: Int = 1

### Pure Functions and Side Effects

Pure functions are functions:

1. Whose return values are only determined by its inputs.
2. Have no observable side effects.

Example of impure function:

```java
class Person {
	int age = 0;
	
	void incrementAge(years:Integer) {
	  this.age += years;
	}
}
```

Example of pure function:

In [56]:
def incrementAge(age:Int, years:Int) = {
	age + years;
}

defined function incrementAge

Reading from the keyboard and writing to a display are considered side effects. The same applies for http requests and any other kind of I/O operation.

### Functions as First Class Citizens

Functions are called first class citizens when the language they are implemented allows them to be assigned to variables or stored in data structures.

In [57]:
val f = (x:Int,y:Int) => x * y
f(2,3)

val functionList = List((x:Int, y:Int) => x * y)

f: (Int, Int) => Int = <function2>
res56_1: Int = 6
functionList: List[(Int, Int) => Int] = List(<function2>)

### Higher Order Functions

Higher order functions are functions that receive other functions as parameters and/or return other functions as results.

In [58]:
trait User
def save(user:User):Boolean = true

def saveIfValid(user:User, validations:List[(User)=>Boolean]) = {
	if (validations.forall(v => v(user))) {
		save(user)
	}
}

def doOperation(opType:String):(Int,Int)=>Int = {
  if (opType == "Sum")
    (a:Int,b:Int) => a + b
  else
    (a:Int,b:Int) => a * b
}

doOperation("Sum")(1,2)
doOperation("Mul")(1,2)

defined trait User
defined function save
defined function saveIfValid
defined function doOperation
res57_4: Int = 3
res57_5: Int = 2

### Immutability

Immutability means that once a variable is assigned with a value of a given type, its value will never change. This is true for both scalar types and other data structures like sequences, sets and maps.

Instead of changing the value of a variable in-place, expressions using immutable variables generate a new value as the result of applying the expression to the variable.

Simple duplicating data structures like lists or vectors, could incurr in memory issues like stack overflow. 

In order to overcome this kind of issues, techniques like persistent data structures, which reuse part of the original data structures, are used.

In [85]:
val x = 1

val x = 2

: 

In [86]:
val aList = List(1,2,3)

val anotherList = aList.map(_*10)

aList: List[Int] = List(1, 2, 3)
anotherList: List[Int] = List(10, 20, 30)

### Closures

A closure is a function together with a referencing environment for the non-local variables of that function. A closure allows a function to access variables outside its immediate lexical scope.

A closure is a block of code which meets three criteria:

1. The block of code can be passed around as a value, and
2. It can be executed on demand by anyone who has that value, at which time, and
3. It can refer to variables from the context in which it was created (i.e. it is closed with respect to variable access, in the mathematical sense of the word “closed”).

In [87]:
val belowFirst = (xs: List[Int]) => { 
    val first = xs(0)
 
    val isBelow = (y: Int) => y < first
 
    for {
        x <- xs;
        if(isBelow(x))
        } yield x
}

belowFirst(List(5, 1, 7, 4, 9, 11, 3))

belowFirst: List[Int] => List[Int] = <function1>
res86_1: List[Int] = List(1, 4, 3)

### Recursion

Recursion is the functional alternative to for loops. Depending on the implementation, recursion can be more efficient than traditional loops.

In [60]:
def factorial(n: Int): Int = {
  if (n == 0) 
    return 1
  else
    return n * factorial(n-1)
}
factorial(5)

defined function factorial
res59_1: Int = 120

### Lazy Sequences

Lazy sequences are sequences whose items are evaluated on demand. It allows us to work with very large sequences and even infinite sequences without raising stack overflow errors.

In [61]:
List("a", "b", "c") zip (Stream from 1)

res60: List[(String, Int)] = List(("a", 1), ("b", 2), ("c", 3))

A lazy factorial implementation using an infinite sequence.

In [62]:
def factorial: Stream[BigInt] = {
  1 #:: (factorial zip Stream.from(1)).map(x => x._1 * x._2)
}
factorial(5)

defined function factorial
res61_1: BigInt = 120

### Functional Abstractions

* Map - Consists in creating a new sequence by applying a function to each item from the original sequence.

In [63]:
(1 to 10).map(_ * 2)

res62: collection.immutable.IndexedSeq[Int] = Vector(2, 4, 6, 8, 10, 12, 14, 16, 18, 20)

* FlatMap - Similar to map, but the function to be applied must return a sequence, instead of a scalar (single value). It will result in a sequence of sequences which are then flattened into a single list.

In [64]:
(1 to 10).flatMap(0 to _)

res63: collection.immutable.IndexedSeq[Int] = Vector(
  0,
  1,
  0,
  1,
  2,
  0,
  1,
  2,
  3,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
...

* Fold - As in map, applies a function to each item of a sequence. The function to be applied must accept a second parameter (called seed) of any type, and return a value of this very same type. The result of the function is used as the seed for the subsequent calls.

In [65]:
(1 to 10).foldLeft(Nil:List[Int])((a,b) => b :: a)

res64: List[Int] = List(10, 9, 8, 7, 6, 5, 4, 3, 2, 1)

* Reduce - A specialization of fold, where the seed type has the same type of the items in the sequence, and the result is  a single value "reduced" by the given function.

In [66]:
(1 to 10).reduce(_+_)

res65: Int = 55

## Advanced Concepts

### Tail Recursion

In order for a recursive call to be tail recursive, the call back to the function must be the last action performed in the function.

A non tail recursive function needs to save intermediate results for each nested function call, which can cause stack overflow errors if there are many nested calls.

First a non tail recursive function:

In [67]:
def factorial(number:Int) : Int = {
    if (number == 1)
       return 1
    number * factorial (number - 1)
}
factorial(5)

defined function factorial
res66_1: Int = 120

Now, a tail recursive implementation:

In [68]:
def factorial(number: Int, accumulator: Int = 1) : Int = {
  if(number == 1)
    accumulator
  else
    factorial(number - 1, number * accumulator)
}
factorial(5)

defined function factorial
res67_1: Int = 120

### Pattern Matching

Pattern matching is a very powerful feature which allows code to be written in a more elegant manner, specially when using concomitantly with recursion.

In [69]:
def factorial(number: Int, accumulator: Int = 1) : Int = number match {
    case 1 => accumulator
    case _ => factorial(number - 1, number * accumulator)
}
factorial(5)

defined function factorial
res68_1: Int = 120

Here's an example of pattern matching on Lists.

In [70]:
def foldLeft[A, B](l: List[A], z: B)(f: (B, A) => B): B = l match {
    case Nil => z
    case (x :: xs) => foldLeft(xs, f(z, x))(f)
}

defined function foldLeft

### Algebraic Data Types

An algebraic data type is a kind of composite type, i.e., a type formed by combining other types.

Two common classes of algebraic types are product types (i.e., tuples and records) and sum types.

Examples of product types are tuples and classes:

```
type Person = (String, Int)
```

```scala
class ScalaPerson(val name: String, val age: Int)
```

Here's an example of a sum type achieved by using subclassing:

In [71]:
trait Color

sealed trait Pet
case class Cat(name: String) extends Pet
case class Fish(name: String, color: Color) extends Pet
case class Squid(name: String, age: Int) extends Pet

val bob: Pet = Cat("Bob")

defined trait Color
defined trait Pet
defined class Cat
defined class Fish
defined class Squid
bob: $user.Pet = Cat(Bob)

Values of algebraic types are analyzed with pattern matching, which identifies a value by its constructor or field names and extracts the data it contains.

In [72]:
def sayHi(p: Pet): String = p match {
    case Cat(n)      => "Meow " + n + "!"
    case Fish(n, _)  => "Hello fishy " + n + "."    
    case Squid(n, _) => "Hi " + n + "."    
}

sayHi(Cat("Bob"))

sayHi(Squid("Steve", 10))

defined function sayHi
res71_1: String = "Meow Bob!"
res71_2: String = "Hi Steve."

Lists are also an algebraic data type of the type sum type. That's why we can use pattern matching to destructure a list by using its constructors :: (Cons) and Nil.

### Monoids and Monads

#### Monoids

##### Definition

A monoid is "a bunch of things" and "some way of combining them".

For example: 

* "integers under addition" is a monoid.
* "strings (or lists) under concatenation" is a monoid.

To be considered a monoid, the combining operation must met the following rules:

1. Rule 1 (Closure): The result of combining two things is always another one of the things. Ex: 1 + 2 = 3
2. Rule 2 (Associativity): When combining more than two things, which pairwise combination you do first doesn't matter. Ex: (1 + 2) + 3 = 1 + (2 + 3)
3. Rule 3 (Identity element): There is a special thing called "zero" such that when you combine any thing with "zero" you get the original thing back. Ex: 3 * 1 = 3 or "test" + "" = "test"

##### Benefits

1. Closure: The closure rule has the benefit that you can convert pairwise operations into operations that work on lists or sequences. In other words, if we can define a pairwise operation, we can extend it to list operations "for free".

The function that does this is typically called "reduce".

In [73]:
List(1,2,3,4).reduce(_+_)

res72: Int = 10

You can see that reduce can be thought of as inserting the specified operation between each element of the list.

2. Associativity: If the pairwise combinations can be done in any order, that opens up some interesting implementation techniques, such as:

* Divide and conquer algorithms
* Parallelization
* Incrementalism

3. Identity: Having an identity element is not always required. Having a closed and associative operation is sufficient to do many useful things. However, in some cases, it is not enough. For example, here are some cases that might crop up:

* How can I use reduce on an empty list?
* If I am designing a divide and conquer algorithm, what should I do if one of the "divide" steps has nothing in it?
* When using an incremental algorithm, what value should I start with when I have no data?

In all cases we need a "zero" value. This allows us to say, for example, that the sum of an empty list is 0.

##### Conclusion

It's important to understand that it's possible to create our own monoidic types for real life entities like Orders, Customers, Products and so on.

By doing that, we can leverage the builtin operations that already know how to work with monoids. From just simplifying our code to enabling parallelism, monoids are a powerful feature of functional programming languages.

#### Monads

##### Definition

One can think about monads **like** being:

* A wrapper for an object.
* An interface with two operations to be implemented.
* A parametric type.
* Monad is not a class or a trait; monad is a concept. Every “wrapper” that provides us with our two operations, unit and flatMap, is essentially a monad.

The two functions a monad must implemented are called:

* identity (or unit)
* bind (or flatMap)

The unit function simply performs the wrapping part, i.e., encapsulates the object in a monad of the object's type.

In [74]:
val listMonad = List(1)
val optionMonad = Some(1)
val setMonad = Set(1)

listMonad: List[Int] = List(1)
optionMonad: Some[Int] = Some(1)
setMonad: Set[Int] = Set(1)

The flatMap function allows us to transform the object wrapped by a monad. The result of applying flatMap is also a monad of the same type.

In [75]:
listMonad.flatMap((x) => List(x+1))
optionMonad.flatMap((x)=>Some(x+1))

res74_0: List[Int] = List(2)
res74_1: Option[Int] = Some(2)

For completeness, it's necessary to mention that in order to be considered a monad, the parametric type needs to conform with the following laws:

* **lef-identity law:** 

```scala
unit(x).flatMap(f) == f(x)
```

* **right-identity law:**

```scala
m.flatMap(unit) == m
```

* **associativity law:**

```scala
m.flatMap(f).flatMap(g) == m.flatMap(x ⇒ f(x).flatMap(g))
```

##### Benefits

Monads help avoiding boilerplate code and also enables composition by removing the need for things like null checks and exception handling.

Here's an example of a program which takes advantage of the Option monad to avoid null checks and exception handling:

In [76]:
trait User {
  val child: Option[User]
}

case class SomeUser (child:Option[User]) extends User

object UserService {
  def loadUser(name: String): Option[User] = None
}

val getChild = (user: User) => user.child

val result = UserService.loadUser("mike")
  .flatMap(getChild)
  .flatMap(getChild)

defined trait User
defined class SomeUser
defined object UserService
getChild: $user.User => Option[$user.User] = <function1>
result: Option[$user.User] = None

When using monads, some languages will provide you with some syntactic sugar to operate on them:

In [77]:
val result = for {
  user             <- UserService.loadUser("mike")
  usersChild       <- user.child
  usersGrandChild  <- usersChild.child
} yield usersGrandChild

result: Option[User] = None

Another application of monads, now using the Future monad. Let's start with the domain model and services:

In [78]:
// needed for Futures to work
import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global

trait Order
case class SomeOrder() extends Order
trait Item
case class SomeItem() extends Item
trait PurchaseResult
case class SomePurchaseResult() extends PurchaseResult
trait LogResult
case class SomeLogResult() extends LogResult

object OrderService {  
  def loadOrder(username: String): Future[Order] = Future(SomeOrder())
}
object ItemService {  
  def loadItem(order: Order): Future[Item] = Future(SomeItem())
}
object PurchasingService { 
  def purchaseItem(item: Item): Future[PurchaseResult] = Future(SomePurchaseResult())
  def logPurchase(purchaseResult: PurchaseResult): Future[LogResult] = Future(SomeLogResult())
}

import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global
defined trait Order
defined class SomeOrder
defined trait Item
defined class SomeItem
defined trait PurchaseResult
defined class SomePurchaseResult
defined trait LogResult
defined class SomeLogResult
defined object OrderService
defined object ItemService
defined object PurchasingService

Now let's create some functions to call our services:

In [79]:
val loadItem: Order => Future[Item] = {
  order => ItemService.loadItem(order)
}
val purchaseItem: Item => Future[PurchaseResult] = {
  item => PurchasingService.purchaseItem(item)
}

val logPurchase: PurchaseResult => Future[LogResult] = {
  purchaseResult => PurchasingService.logPurchase(purchaseResult)
}

loadItem: Order => Future[Item] = <function1>
purchaseItem: Item => Future[PurchaseResult] = <function1>
logPurchase: PurchaseResult => Future[LogResult] = <function1>

Finally, let's chain everything together.

In [80]:
val result = 
  OrderService.loadOrder("customerUsername")
  .flatMap(loadItem)
  .flatMap(purchaseItem)
  .flatMap(logPurchase)

result: Future[LogResult] = List()

Once again, by using monads we can leverage the for comprehension syntactic sugar (in Scala).

In [81]:
val result =
  for {
    loadedOrder    <- OrderService.loadOrder("customerUsername")
    loadedItem     <- ItemService.loadItem(loadedOrder)
    purchaseResult <- PurchasingService.purchaseItem(loadedItem)
    logResult      <- PurchasingService.logPurchase(purchaseResult)
  } yield logResult

result: Future[LogResult] = Success(SomeLogResult())

##### Conclusion

Monads with its two operations, unit and flatMap, is a pretty powerful feature. Forget entangled if-branches, nested loops with their off-by-one errors and the callback hell. Monads enable you to write simpler and cleaner code, which often leads to less bugs in production.

Just like monoids, beyond using the existing monads, we can create our own in order to make our codebase simpler and more elegant.

<center> **Thank You!** </center>